In [1]:
#Task:
#Take any csv/text file of 2+ GB of your choice. --- (You can do this assignment on Google colab)

#Read the file ( Present approach of reading the file )

#Try different methods of file reading eg: Dask, Modin, Ray, pandas and present your findings in term of computational efficiency

#Perform basic validation on data columns : eg: remove special character , white spaces from the col name

#As you already know the schema hence create a YAML file and write the column name in YAML file. --define separator of
#read and write file, column name in YAML

#Validate number of columns and column name of ingested file with YAML.

#Write the file in pipe separated text file (|) in gz format.

#Create a summary of the file:

#Total number of rows,

#total number of columns

#file size

In [2]:
# Take any csv/text file of 2+GB of your choice
# I will start by  creating my own huge-sized dataframe with 20000000(20 million ) rows and 14 columns.

In [3]:
import pandas as pd
import numpy as np

In [4]:
df = pd.DataFrame(data=np.random.randint(99999, 99999999, size=(20000000,14)),columns=['C1','C2','C3','C4','C5','C6','C7','C8','C9','C10','C11','C12','C13','C14'])

In [5]:
# The dataset contains 14 string columns with record of random strings.

In [6]:
df.shape

(20000000, 14)

In [7]:
df.head()

,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14
0,88969219,89690489,85762767,40268439,86762717,53725854,30717872,81810106,94213974,76111916,33102367,88778704,18177532,93274930
1,42109376,13441416,28385810,7208095,6593260,70358103,51482372,84673699,73867756,24404078,75343551,57158857,22455626,17871407
2,91733537,46567399,95799279,90452316,9546710,88432699,23387102,88906698,93573526,10402492,70555685,98749264,89520506,53985279
3,77196862,61045313,67290184,66928455,97201015,24743320,19636486,87273370,2846256,36564591,68390890,94682207,1395493,52751425
4,46177319,13104254,34592816,27305311,57539064,42976395,28598586,43182383,4689554,94100852,57642818,6515197,49330612,60674126


In [8]:
# We need to store the data in CSV format
df.to_csv("huge_data.csv",index=False)

In [8]:
%%writefile testutility.py
import logging
import os
import subprocess
import yaml
import pandas as pd
import datetime 
import gc
import re


################
# File Reading #
################

def read_config_file(filepath):
    with open(filepath, 'r') as stream:
        try:
            return yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            logging.error(exc)


def replacer(string, char):
    pattern = char + '{2,}'
    string = re.sub(pattern, char, string) 
    return string

def col_header_val(df,table_config):
    '''
    replace whitespaces in the column
    and standardized column names
    '''
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace('[^\w]','_',regex=True)
    df.columns = list(map(lambda x: x.strip('_'), list(df.columns)))
    df.columns = list(map(lambda x: replacer(x,'_'), list(df.columns)))
    expected_col = list(map(lambda x: x.lower(),  table_config['columns']))
    expected_col.sort()
    df.columns =list(map(lambda x: x.lower(), list(df.columns)))
    df = df.reindex(sorted(df.columns), axis=1)
    if len(df.columns) == len(expected_col) and list(expected_col)  == list(df.columns):
        print("column name and column length validation passed")
        return 1
    else:
        print("column name and column length validation failed")
        mismatched_columns_file = list(set(df.columns).difference(expected_col))
        print("Following File columns are not in the YAML file",mismatched_columns_file)
        missing_YAML_file = list(set(expected_col).difference(df.columns))
        print("Following YAML columns are not in the file uploaded",missing_YAML_file)
        logging.info(f'df columns: {df.columns}')
        logging.info(f'expected columns: {expected_col}')
        return 0

Overwriting testutility.py


In [9]:
%%writefile file.yaml
file_type: csv
dataset_name: hugefile
file_name: huge_data
table_name: edsurv
inbound_delimiter: ","
outbound_delimiter: "|"
skip_leading_rows: 1
columns: 
    - C14
    - C15
    - C16

Overwriting file.yaml


In [10]:
# Read config file
import testutility as util
config_data = util.read_config_file("file.yaml")

In [11]:
config_data['inbound_delimiter']

','

In [12]:
#inspecting data of config file
config_data

{'file_type': 'csv',
 'dataset_name': 'hugefile',
 'file_name': 'huge_data',
 'table_name': 'edsurv',
 'inbound_delimiter': ',',
 'outbound_delimiter': '|',
 'skip_leading_rows': 1,
 'columns': ['C14', 'C15', 'C16']}

In [13]:
# Normal reading process of the file
import pandas as pd
df_sample = pd.read_csv("huge_data.csv",delimiter=',')
df_sample.head()

,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14
0,26273389,60601112,29221815,1416189,75314224,49176151,43130315,53149488,25336470,90051076,34806491,29170887,66146855,23005957
1,4918494,9194009,78959634,78737439,65704896,32451169,72612838,61095012,43156405,15568099,8000124,88893415,76234474,76731031
2,6269183,96262737,74869103,44073475,38863162,36216575,59620142,75983890,4748715,59078507,47491177,37189488,32678775,88675361
3,47144088,11727956,80480525,10523266,4741759,11328663,70280314,98468484,76187140,9118630,5214450,74775317,37429831,54974609
4,43573099,16615868,38826553,16263299,50898322,60338156,5082484,28830650,93134435,49207920,6658803,72340193,9066401,13437986


In [15]:
# The size of the file
import os
print(os.path.getsize('huge_data.csv')/1024/1024/1024 , "GB")

2.337380526587367 GB


In [14]:
# Reading csv without chunks.
import time
import pandas as pd
start = time.time()
df = pd.read_csv('huge_data.csv')
end = time.time()
print("Read csv without chunks: ",(end-start),"sec")

Read csv without chunks:  850.6296706199646 sec


In [16]:
%%time
df.iloc[:,1].mean()

Wall time: 8 s


50046008.58555365

In [17]:
# Reading csv with chunks.
start = time.time()
chunk = pd.read_csv('huge_data.csv',chunksize=1000000)
end = time.time()
print("Read csv with chunks: ",(end-start),"sec")
pd_df = pd.concat(chunk)

Read csv with chunks:  0.9371664524078369 sec


In [18]:
type(chunk)

pandas.io.parsers.TextFileReader

In [19]:
type(pd_df)

pandas.core.frame.DataFrame

In [20]:
%%time
pd_df.iloc[:,1].mean()

Wall time: 7.78 s


50046008.58555365

In [21]:
from dask import dataframe as dd
from dask.distributed import Client

C:\Users\Nkululeko\Anaconda3\lib\site-packages\distributed\utils.py:133: RuntimeWarning: Couldn't detect a suitable IP address for reaching '8.8.8.8', defaulting to '127.0.0.1': [WinError 10051] A socket operation was attempted to an unreachable network
  RuntimeWarning,


In [22]:
#dask.dataframe.read_csv()¶
#Start Dask Client for Dashboard
#It will provide a dashboard which is useful to gain insight on the computation.
client = Client(processes=False)
client

Client Scheduler: inproc://127.0.0.1/13392/1 Dashboard: http://127.0.0.1/13392/1:8787/status,Cluster Workers: 1 Cores: 2 Memory: 4.11 GB


In [23]:
# Read csv with dask
start = time.time()
dask_df = dd.read_csv('huge_data.csv')
end = time.time()
print("Read csv with dask: ",(end-start),"sec")

Read csv with dask:  8.047312021255493 sec


In [24]:
%%time
dask_df.iloc[:,1].mean().compute()

Wall time: 2min 6s


50046008.58555365

In [25]:
# Dask performed much faster while reading large CSV prepared.
#Even pandas can perform better while loading up in chunks via chunksize argument.

In [26]:
# read the file using config file
file_type = config_data['file_type']
source_file = "./" + config_data['file_name'] + f'.{file_type}'
#print("",source_file)
df = pd.read_csv(source_file,config_data['inbound_delimiter'])
df.head()

,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14
0,26273389,60601112,29221815,1416189,75314224,49176151,43130315,53149488,25336470,90051076,34806491,29170887,66146855,23005957
1,4918494,9194009,78959634,78737439,65704896,32451169,72612838,61095012,43156405,15568099,8000124,88893415,76234474,76731031
2,6269183,96262737,74869103,44073475,38863162,36216575,59620142,75983890,4748715,59078507,47491177,37189488,32678775,88675361
3,47144088,11727956,80480525,10523266,4741759,11328663,70280314,98468484,76187140,9118630,5214450,74775317,37429831,54974609
4,43573099,16615868,38826553,16263299,50898322,60338156,5082484,28830650,93134435,49207920,6658803,72340193,9066401,13437986


In [27]:
#validate the header of the file
util.col_header_val(df,config_data)

column name and column length validation failed
Following File columns are not in the YAML file ['c5', 'c3', 'c10', 'c1', 'c7', 'c12', 'c4', 'c8', 'c6', 'c13', 'c2', 'c9', 'c11']
Following YAML columns are not in the file uploaded ['c16', 'c15']


0

In [28]:
print("columns of files are:" ,df.columns)
print("columns of YAML are:" ,config_data['columns'])

columns of files are: Index(['c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9', 'c10', 'c11',
       'c12', 'c13', 'c14'],
      dtype='object')
columns of YAML are: ['C14', 'C15', 'C16']


In [29]:
if util.col_header_val(df,config_data)==0:
    print("validation failed")
    # write code to reject the file
else:
    print("col validation passed")
    # write the code to perform further action
    # in the pipleine

column name and column length validation failed
Following File columns are not in the YAML file ['c5', 'c3', 'c10', 'c1', 'c7', 'c12', 'c4', 'c8', 'c6', 'c13', 'c2', 'c9', 'c11']
Following YAML columns are not in the file uploaded ['c16', 'c15']
validation failed


In [30]:
pd.read_csv("./huge_data.csv")

distributed.comm.inproc - WARNING - Closing dangling queue in <InProc  local=inproc://127.0.0.1/13392/10 remote=inproc://127.0.0.1/13392/1>


,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14
0,26273389,60601112,29221815,1416189,75314224,49176151,43130315,53149488,25336470,90051076,34806491,29170887,66146855,23005957
1,4918494,9194009,78959634,78737439,65704896,32451169,72612838,61095012,43156405,15568099,8000124,88893415,76234474,76731031
2,6269183,96262737,74869103,44073475,38863162,36216575,59620142,75983890,4748715,59078507,47491177,37189488,32678775,88675361
3,47144088,11727956,80480525,10523266,4741759,11328663,70280314,98468484,76187140,9118630,5214450,74775317,37429831,54974609
4,43573099,16615868,38826553,16263299,50898322,60338156,5082484,28830650,93134435,49207920,6658803,72340193,9066401,13437986
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19999995,67797999,31343222,68847487,19367762,42881347,76526618,6384575,1434200,71243384,83238816,18929636,10303297,50639510,9628783
19999996,8249562,18935693,21614455,23260712,18287203,35427787,88951097,37457391,1278276,10656870,98763287,60189145,83306016,38997594
19999997,96036501,20851136,38206060,8942692,95875382,51279032,98224993,89714651,20875755,55446188,19520131,5707837,92780455,88832748
19999998,67989279,46072392,48402215,75983151,93141172,39346147,92170750,79740018,84917340,55952285,80066605,20526820,21412530,88615818


In [31]:
df

,c1,c2,c3,c4,c5,c6,c7,c8,c9,c10,c11,c12,c13,c14
0,26273389,60601112,29221815,1416189,75314224,49176151,43130315,53149488,25336470,90051076,34806491,29170887,66146855,23005957
1,4918494,9194009,78959634,78737439,65704896,32451169,72612838,61095012,43156405,15568099,8000124,88893415,76234474,76731031
2,6269183,96262737,74869103,44073475,38863162,36216575,59620142,75983890,4748715,59078507,47491177,37189488,32678775,88675361
3,47144088,11727956,80480525,10523266,4741759,11328663,70280314,98468484,76187140,9118630,5214450,74775317,37429831,54974609
4,43573099,16615868,38826553,16263299,50898322,60338156,5082484,28830650,93134435,49207920,6658803,72340193,9066401,13437986
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19999995,67797999,31343222,68847487,19367762,42881347,76526618,6384575,1434200,71243384,83238816,18929636,10303297,50639510,9628783
19999996,8249562,18935693,21614455,23260712,18287203,35427787,88951097,37457391,1278276,10656870,98763287,60189145,83306016,38997594
19999997,96036501,20851136,38206060,8942692,95875382,51279032,98224993,89714651,20875755,55446188,19520131,5707837,92780455,88832748
19999998,67989279,46072392,48402215,75983151,93141172,39346147,92170750,79740018,84917340,55952285,80066605,20526820,21412530,88615818
